In [1]:
import requests
import os
import pandas as pd
import time
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


def sis_import(filnamn):
    url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
    filar = {'attachment': open(filnamn, 'rb')}
    r = requests.post(url, headers=headers, files=filar)
    resultat = r.json()
    vent = True
    forseinking = 5
    while vent:
    # Her venter eg til alt er ferdig før eg tar neste campus
        jobb_id = resultat['id']
        time.sleep(forseinking)
        url = f"https://hvl.instructure.com/api/v1/accounts/1/sis_imports/{jobb_id}"
        r = requests.get(url, headers=headers)
        resultat = r.json()
        if resultat['progress']==100:
            print("Ferdig.")
            vent = False
        else:
            print(f"Arbeider framleis, {resultat['progress']} % ferdig.")

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


# Avslutte manuelt innlagt i eldre emne

## Problemstilling
Når vi legg inn brukarar manuelt i emne vert dei liggande aktive etter at emneet er avslutta, for det vert ikkje sendt melding om å avslutte innmeldinga for desse, slik det vert gjort for dei som kjem frå FS.

## Løysing
Eg reknar alle emne som har `enrollment_term_id` i lista over `avslutt_semestre` som avslutta. Det er desse som får "avslutt"-melding frå FS våren 2025 (kanskje). 
Så går eg gjennom denne lista (ca. 16000 emne) og finn alle personar som framleis er aktive. 

## CD2
Eg henter først alle `courses` frå CD2, for å sjå kva eg får ut.

In [9]:
alle_emne = pd.read_csv("alle_emne_250624.csv")

Så må eg finne alle terminane vi har, slik at eg kan plukke ut alle emne som skal avsluttast (emne som 30.6.2025 har "2024 VÅR" som siste termin). Så kan eg jo gjere dette for andre semestre også, men det seinare.

In [10]:
url = "https://hvl.instructure.com/api/v1/accounts/1/terms"
dr_liste = []
hentmeir = True
while hentmeir:
    print(url)
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
        dataliste = []
# Den neste linja varierer frå endepunkt til endepunkt
        for element in data['enrollment_terms']:
            dataliste.append(element)
        df = pd.DataFrame(dataliste, columns=['id', 'name'])
        dr_liste.append(df)
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

avslutt = alledata[alledata['name'].str.endswith('2022 HØST')]['id']

https://hvl.instructure.com/api/v1/accounts/1/terms
https://hvl.instructure.com/api/v1/accounts/1/terms?page=2&per_page=100


Så kan eg leite etter emne som høyrer til i kvar termin:

In [11]:
avslutt_emne = []
for termin in avslutt:
    avslutt_emne.append(alle_emne[alle_emne['value.enrollment_term_id'] == termin]['key.id'].to_list())
    print(f"Emne som skal avsluttast for termin {termin}: {len(alle_emne[alle_emne['value.enrollment_term_id'] == termin])}")

Emne som skal avsluttast for termin 39: 2
Emne som skal avsluttast for termin 86: 1
Emne som skal avsluttast for termin 93: 3
Emne som skal avsluttast for termin 199: 5
Emne som skal avsluttast for termin 204: 19
Emne som skal avsluttast for termin 208: 1160


In [12]:
deaktiver = []
for termin in avslutt_emne:
    print(f"Termin: {termin}")
    for emnekode in termin:  # Endre indeksen for å velge riktig termin
        try:
            print(f"  Emne: {emnekode}")
            url = f"https://hvl.instructure.com/api/v1/courses/{emnekode}/sections"
            seksjonar = requests.get(url, headers=headers, params=params).json()
            for s in seksjonar:
                print(f"    Seksjon: {s['id']}")
                if s['sis_section_id'] == None:
                    print(f"    Lager sis_section_id for {s['id']}")
                    response = requests.put(f"https://hvl.instructure.com/api/v1/sections/{s['id']}",
                                headers=headers, params={"course_section[sis_section_id]": f"hvl250619-{s['id']}-akv"})
            
            url = f"https://hvl.instructure.com/api/v1/courses/{emnekode}/enrollments"
            innmeldingar = requests.get(url, headers=headers, params=params).json()
            print(f"  {emnekode}", end="")
            manglar_seksjon = False
            for i in innmeldingar:
                if i['sis_section_id'] == None:
                    manglar_seksjon = True
                if i['enrollment_state'] == 'active':
                    if i['sis_user_id'] != None:
                        if i['role'] == "StudentEnrollment":
                            deaktiver.append([i['sis_user_id'],i['sis_section_id'],"Student","completed"])
                        if i['role'] == "TeacherEnrollment":
                            deaktiver.append([i['sis_user_id'],i['sis_section_id'],"Teacher","completed"])
            if manglar_seksjon:
                print(" - manglar seksjon")
            else:
                print(" - OK")
        except:
            print(f"  Feil ved {emnekode}")
pd.DataFrame(deaktiver, columns=['user_id', 'section_id', 'role', 'status']).to_csv("deaktiver.csv", index=False)

Termin: [7727, 9005]
  Emne: 7727
    Seksjon: 8086
  7727 - OK
  Emne: 9005
    Seksjon: 11045
    Seksjon: 18489
    Seksjon: 22647
  9005 - OK
Termin: [13038]
  Emne: 13038
    Seksjon: 16843
  13038 - OK
Termin: [13777, 14211, 16285]
  Emne: 13777
    Seksjon: 17470
  13777 - OK
  Emne: 14211
    Seksjon: 17939
  14211 - OK
  Emne: 16285
    Seksjon: 21582
    Seksjon: 26480
    Lager sis_section_id for 26480
  16285 - OK
Termin: [17365, 17373, 17388, 17793, 18984]
  Emne: 17365
    Seksjon: 22621
    Seksjon: 23678
    Lager sis_section_id for 23678
  17365 - OK
  Emne: 17373
    Seksjon: 22763
    Seksjon: 23677
    Lager sis_section_id for 23677
  17373 - OK
  Emne: 17388
    Seksjon: 22774
    Seksjon: 23689
    Lager sis_section_id for 23689
  17388 - OK
  Emne: 17793
    Seksjon: 23029
    Seksjon: 24019
    Lager sis_section_id for 24019
  17793 - OK
  Emne: 18984
    Seksjon: 25810
    Seksjon: 32851
    Lager sis_section_id for 32851
  18984 - OK
Termin: [18175, 18213, 182

In [36]:
sis_import("deaktiver.csv")

Arbeider framleis, 0 % ferdig.
Arbeider framleis, 0 % ferdig.
Arbeider framleis, 0 % ferdig.
Arbeider framleis, 0 % ferdig.
Ferdig.
